In [1]:
from collections import OrderedDict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import flwr as fl
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

from datasets import Dataset
from flwr_datasets.partitioner import DirichletPartitioner

from custom_flwr.server_app import server_fn as server_fn_custom
from custom_flwr.client_app import client_fn as client_fn_custom

DEVICE = torch.device('cpu')

NUM_CLIENTS = 10
# REPRESENTS SILO'D ORGANIZATIONS

/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-06 11:57:21,986	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:


def server_fn(context: Context):
    context.run_config = {
        'num-server-rounds' : 10,
        'fraction-fit': 0.25,
        'fraction-evaluate': 0.5,
        'local-epochs': 1,
        'server-device': str(DEVICE),
        'use-wandb': False
    }
    return server_fn_custom(context)

def client_fn(context: Context):
    return client_fn_custom(context)

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)

In [ ]:
backend_config = {"client_resources": None}
NUM_PARTITIONS = 10
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)